In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import xgboost as xgb
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from xgboost import plot_importance


%matplotlib inline

In [4]:
gm_10 = pd.read_csv("/content/drive/MyDrive/YSAL/2023-1/LOL/GM10_fix.csv")
gm_15 = pd.read_csv("/content/drive/MyDrive/YSAL/2023-1/LOL/GM15_fix.csv")

In [5]:
drop_gm10 = gm_10.drop(['Unnamed: 0', 'gameId','redWins',
                   "blueFirstTowerLane",'blueDragonType','redFirstTowerLane','redDragonType'], axis=1)
drop_gm15 = gm_15.drop(['Unnamed: 0', 'gameId','redWins',
                   "blueFirstTowerLane",'blueDragonType','redFirstTowerLane','redDragonType'], axis=1)

In [7]:
def train_test(df):
  y_train = df['blueWins']
  x_train = df.drop(['blueWins'], axis=1)
  return x_train, y_train

In [8]:
x_train, y_train = train_test(drop_gm15)

val_size=0.3

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = val_size, random_state=42)

# Xgboost

In [9]:
pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from bayes_opt import BayesianOptimization

In [11]:
# 탐색 대상 함수 (XGBRegressor)
def XGB_cv(max_depth,learning_rate, n_estimators, gamma
            ,min_child_weight, subsample
            ,colsample_bytree, silent=True, nthread=-1):

    # 모델 정의
    model = xgb.XGBClassifier(max_depth=int(max_depth),
                            learning_rate=learning_rate,
                            n_estimators=int(n_estimators),
                            gamma=gamma,
                            min_child_weight=min_child_weight,
                            subsample=subsample,
                            colsample_bytree=colsample_bytree, 
                            nthread=nthread
                            )
    # 모델 훈련
    model.fit(x_train, y_train)

    # 예측값 출력
    y_pred= model.predict(x_valid)

    return f1_score(y_valid, y_pred)

In [20]:
pbounds = {'max_depth': (3, 7),
                'learning_rate': (0.01, 0.3),
                'n_estimators': (20,100),
                'gamma': (0, 100),
                'min_child_weight': (0, 3),
                'subsample': (0.5, 1),
                'colsample_bytree' :(0.2, 1)
                }

xg_bo=BayesianOptimization(f=XGB_cv, pbounds=pbounds, verbose=2, random_state=1 )    

xg_bo.maximize(init_points=2, n_iter=100)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
| 1         | 0.6317    | 0.5336    | 72.03     | 0.01003   | 4.209     | 0.4403    | 27.39     | 0.5931    |
| 2         | 0.6855    | 0.4764    | 39.68     | 0.1663    | 4.677     | 2.056     | 36.36     | 0.9391    |
| 3         | 0.6409    | 0.2046    | 40.19     | 0.1041    | 3.836     | 2.294     | 37.26     | 0.6836    |
| 4         | 0.6938    | 0.8121    | 10.95     | 0.1614    | 6.073     | 2.061     | 30.64     | 0.601     |
| 5         | 0.6778    | 0.972     | 6.63      | 0.02515   | 5.446     | 1.737     | 88.86     | 0.7013    |
| 6         | 0.6468    | 0.4585    | 71.28     | 0.1492    | 3.107     | 2.048     | 82.37     | 0.987     |
| 7         | 0.652     | 0.5531    | 50.9      | 0.1793    | 6.358     | 0.008091  | 85.44     | 0.5556    |
| 8       

# Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
def bo_params_rf(max_samples,n_estimators,max_features, max_depth):
    
    params = {
        'max_samples': max_samples,
        'max_features':max_features,
        'max_depth':int(max_depth),
        'n_estimators':int(n_estimators)
    }
    clf = RandomForestClassifier(**params)
    
    clf.fit(x_train, y_train)

    # 예측값 출력
    y_pred= clf.predict(x_valid)

    return f1_score(y_valid, y_pred)

In [15]:
pbounds = {'max_samples':(0.5,1),
        'max_features':(0.5,1),
        'max_depth' : (3, 7),
        'n_estimators':(10,200)}

rf_bo = BayesianOptimization(bo_params_rf, pbounds=pbounds, verbose=2, random_state=1)
rf_bo.maximize(init_points=2, n_iter=100)

|   iter    |  target   | max_depth | max_fe... | max_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.7806    | 4.668     | 0.8602    | 0.5001    | 67.44     |
| 2         | 0.7687    | 3.587     | 0.5462    | 0.5931    | 75.66     |
| 3         | 0.7781    | 4.922     | 0.756     | 0.6562    | 67.59     |
| 4         | 0.7689    | 3.523     | 0.7864    | 0.6083    | 67.12     |
| 5         | 0.7668    | 4.949     | 0.7672    | 0.7738    | 67.53     |
| 6         | 0.7725    | 3.085     | 0.9039    | 0.9824    | 143.4     |
| 7         | 0.7732    | 4.458     | 0.5112    | 0.5762    | 194.6     |
| 8         | 0.7727    | 6.981     | 0.6626    | 0.9084    | 18.48     |
| 9         | 0.7735    | 6.925     | 0.6352    | 0.9044    | 94.85     |
| 10        | 0.7732    | 4.3       | 0.8179    | 0.7003    | 152.8     |
| 11        | 0.7808    | 4.224     | 0.8383    | 0.7757    | 70.01     |
| 12        | 0.7674    | 4.589     | 

In [21]:
print(xg_bo.max)
print(rf_bo.max)

{'target': 0.6978967495219885, 'params': {'colsample_bytree': 0.3559933667008637, 'gamma': 8.834445524484702, 'learning_rate': 0.2553193067332952, 'max_depth': 5.272480565844037, 'min_child_weight': 1.721934014761535, 'n_estimators': 61.74875769041427, 'subsample': 0.9674259123440412}}
{'target': 0.7818012999071494, 'params': {'max_depth': 6.722808724081828, 'max_features': 0.9562452054707964, 'max_samples': 0.8774713674032177, 'n_estimators': 140.97627307723155}}


# 다른 데이터에 적용

In [17]:
x_train, y_train = train_test(drop_gm10)

val_size=0.3

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = val_size, random_state=42)

xgboost

In [23]:
params = {'colsample_bytree': 0.3559933667008637, 'gamma': 8.834445524484702, 'learning_rate': 0.2553193067332952, 'max_depth': 5, 'min_child_weight': 1.721934014761535, 'n_estimators': 62, 'subsample': 0.9674259123440412}

model = xgb.XGBClassifier(**params)
model.fit(x_train, y_train)
valid_pred = model.predict(x_valid)
print("F1 score: %.2f%%" % (f1_score(y_valid, valid_pred) * 100.0))
print("Accuracy: %.2f%%" % (accuracy_score(y_valid, valid_pred) * 100.0))

F1 score: 69.79%
Accuracy: 72.62%


Rf

In [24]:
params = {'max_depth': 7, 'max_features': 0.9562452054707964, 'max_samples': 0.8774713674032177, 'n_estimators': 141}

clf = RandomForestClassifier(**params)
model.fit(x_train, y_train)
valid_pred = model.predict(x_valid)
print("F1 sore: %.2f%%" % (f1_score(y_valid, valid_pred) * 100.0))
print("Accuracy: %.2f%%" % (accuracy_score(y_valid, valid_pred) * 100.0))

F1 sore: 69.79%
Accuracy: 72.62%
